In [16]:
import random
import numpy as np
from typing import List, Dict, Set, Tuple, Optional
from alns import ALNS, State
from alns.accept import SimulatedAnnealing
from alns.select import RouletteWheel
from alns.stop import MaxIterations
import numpy.random as rnd

SEED = 1234

class Route:
    """
    Represents a van or robot route in the 2E-VREC problem.
    """
    def __init__(self, is_van_route: bool = True):
        self.nodes: List[int] = []  # List of nodes in the route
        self.is_van_route: bool = is_van_route
        # Whether robot is on the van for each segment (van routes only)
        self.robot_onboard: List[bool] = []
        # Amount to recharge at each node
        self.recharge_amount: Dict[int, float] = {}
        # Amount to charge en-route (van routes only)
        self.en_route_charge: Dict[Tuple[int, int], float] = {}

    def copy(self) -> "Route":
        """Creates a deep copy of the Route object."""
        route = Route(self.is_van_route)
        route.nodes = self.nodes.copy()
        route.robot_onboard = self.robot_onboard.copy()
        route.recharge_amount = self.recharge_amount.copy()
        route.en_route_charge = self.en_route_charge.copy()
        return route

    def __str__(self) -> str:
        route_type = "Van" if self.is_van_route else "Robot"
        return f"{route_type} Route: {self.nodes}"

class TwoEVREC(State):
    """
    Represents a solution to the 2E-VREC problem.
    """
    def __init__(self,
                 distance_matrix: np.ndarray,
                 depot: int,
                 charging_stations: List[int],
                 customers_robot_only: List[int],
                 customers_both: List[int],
                 van_params: Dict[str, float],
                 robot_params: Dict[str, float],
                 customer_demand: Dict[int, float],
                 time_windows: Optional[Dict[int, Tuple[float, float]]] = None):

        # Problem data
        self.distance_matrix: np.ndarray = distance_matrix
        self.depot: int = depot
        self.charging_stations: List[int] = charging_stations
        self.customers_robot_only: List[int] = customers_robot_only
        self.customers_both: List[int] = customers_both
        self.all_customers: List[int] = customers_robot_only + customers_both
        self.van_params: Dict[str, float] = van_params
        self.robot_params: Dict[str, float] = robot_params
        self.customer_demand: Dict[int, float] = customer_demand
        self.time_windows: Dict[int, Tuple[float, float]] = time_windows or {}

        # Solution data
        self.van_routes: List[Route] = []
        self.robot_routes: List[Route] = []
        self.unassigned_customers: Set[int] = set(self.all_customers)

    def copy(self) -> "TwoEVREC":
        """Create a deep copy of the solution state."""
        solution = TwoEVREC(
            self.distance_matrix,
            self.depot,
            self.charging_stations,
            self.customers_robot_only,
            self.customers_both,
            self.van_params,
            self.robot_params,
            self.customer_demand,
            self.time_windows
        )

        solution.van_routes = [route.copy() for route in self.van_routes]
        solution.robot_routes = [route.copy() for route in self.robot_routes]
        solution.unassigned_customers = self.unassigned_customers.copy()

        return solution

    def objective(self) -> float:
        """
        Calculate the total cost of the solution: van route cost plus robot route cost,
        minus the van-robot route cost.
        """
        total_cost = 0.0

        # Van route cost
        for route in self.van_routes:
            if len(route.nodes) > 1:  # Only calculate if route has at least 2 nodes
                for i in range(len(route.nodes) - 1):
                    node1, node2 = route.nodes[i], route.nodes[i + 1]
                    total_cost += (self.van_params["travel_cost_rate"] *
                                  self.distance_matrix[node1][node2])

        # Robot route cost
        for route in self.robot_routes:
            if len(route.nodes) > 1:
                for i in range(len(route.nodes) - 1):
                    node1, node2 = route.nodes[i], route.nodes[i + 1]
                    total_cost += (self.robot_params["travel_cost_rate"] *
                                  self.distance_matrix[node1][node2])

        # Subtract van-robot route cost (where robot is onboard)
        for route in self.van_routes:
            if len(route.nodes) > 1:
                for i in range(len(route.nodes) - 1):
                    if i < len(route.robot_onboard) and route.robot_onboard[i]:
                        node1, node2 = route.nodes[i], route.nodes[i + 1]
                        # Subtract robot cost (robot not moving independently)
                        total_cost -= (self.robot_params["travel_cost_rate"] *
                                      self.distance_matrix[node1][node2])

        return total_cost

    def is_feasible(self) -> bool:
        """
        Check if the solution is feasible by verifying:
        - All customers are served
        - Vehicle capacities are respected
        - Energy constraints are respected
        - Time windows are respected
        """
        if self.unassigned_customers:
            return False

        return all(self._is_van_route_feasible(route, i)
                  for i, route in enumerate(self.van_routes)) and \
               all(self._is_robot_route_feasible(route, i)
                  for i, route in enumerate(self.robot_routes))

    def _is_van_route_feasible(self, route: Route, route_idx: int) -> bool:
        """Check if a van route is feasible considering capacity, energy, and time constraints."""
        if not route.nodes:
            return True  # Empty route is feasible

        if route.nodes[0] != self.depot or route.nodes[-1] != self.depot:
            return False  # Route must start and end at depot

        # Initial conditions
        battery_level = self.van_params["battery_capacity"]
        load = 0  # Start with empty van
        current_time = 0  # Start time

        # Use the greedy route evaluation approach
        for i in range(len(route.nodes) - 1):
            node1, node2 = route.nodes[i], route.nodes[i + 1]

            # Check time windows
            if node1 in self.time_windows:
                earliest, latest = self.time_windows[node1]
                if current_time < earliest:
                    current_time = earliest
                elif current_time > latest:
                    return False  # Time window violation

            # Service time at node
            if node1 in self.all_customers:
                service_time = 0.1  # Assumed constant service time
                current_time += service_time

            # Check if robot is dropped off or picked up
            if i > 0 and i < len(route.robot_onboard):
                if not route.robot_onboard[i-1] and route.robot_onboard[i]:
                    # Robot picked up, adjust load
                    load += sum(self.customer_demand.get(n, 0) for n in self.all_customers
                               if n in route.nodes[i:])

            # Pickup customer demand
            if node1 == self.depot:
                load += sum(self.customer_demand.get(n, 0) for n in self.all_customers
                           if n in route.nodes)

            # Delivery, reduce load
            if node1 in self.all_customers and route.robot_onboard[i-1] if i > 0 else False:
                load -= self.customer_demand[node1]

            # Check load capacity
            if load > self.van_params["capacity"]:
                return False  # Capacity violation

            # Recharge at charging station
            if node1 in self.charging_stations:
                recharge_amount = route.recharge_amount.get(node1, 0)
                recharge_time = recharge_amount / self.van_params["charge_rate"]
                current_time += recharge_time
                battery_level = min(battery_level + recharge_amount,
                                   self.van_params["battery_capacity"])

            # Travel to next node
            distance = self.distance_matrix[node1][node2]
            travel_time = distance / self.van_params["speed"]
            current_time += travel_time

            # Energy consumption
            energy_consumption = distance * self.van_params["energy_consumption_rate"]

            # En-route charging (energy transfer to robot)
            en_route_charge = route.en_route_charge.get((node1, node2), 0)
            if en_route_charge > 0:
                battery_level -= en_route_charge  # Transfer energy to robot

            # Energy consumption for travel
            battery_level -= energy_consumption

            if battery_level < 0:
                return False  # Energy depletion

        # Check time window for the last node
        last_node = route.nodes[-1]
        if last_node in self.time_windows:
            earliest, latest = self.time_windows[last_node]
            if current_time < earliest:
                current_time = earliest
            elif current_time > latest:
                return False  # Time window violation

        return True

    def _is_robot_route_feasible(self, route: Route, route_idx: int) -> bool:
        """Check if a robot route is feasible considering capacity, energy, and time constraints."""
        if not route.nodes:
            return True  # Empty route is feasible

        # Robot routes must start and end at charging stations
        if route.nodes[0] not in self.charging_stations or route.nodes[-1] not in self.charging_stations:
            return False

        # Initial conditions
        battery_level = 0  # Will be set after charging at the first station
        load = 0  # Start with empty robot
        current_time = 0  # Will be synchronized with van time later

        for i in range(len(route.nodes) - 1):
            node1, node2 = route.nodes[i], route.nodes[i + 1]

            # Recharge at charging station
            if node1 in self.charging_stations:
                recharge_amount = route.recharge_amount.get(node1, 0)
                recharge_time = recharge_amount / self.robot_params["charge_rate"]
                current_time += recharge_time
                battery_level = min(battery_level + recharge_amount,
                                   self.robot_params["battery_capacity"])

            # Add customer demand to robot
            if node1 in self.all_customers:
                load += self.customer_demand[node1]
                service_time = 0.1  # Assumed constant service time
                current_time += service_time

            # Check capacity
            if load > self.robot_params["capacity"]:
                return False  # Capacity violation

            # Check time windows
            if node1 in self.time_windows:
                earliest, latest = self.time_windows[node1]
                if current_time < earliest:
                    current_time = earliest
                elif current_time > latest:
                    return False  # Time window violation

            # Travel to next node
            distance = self.distance_matrix[node1][node2]
            travel_time = distance / self.robot_params["speed"]
            current_time += travel_time

            # Energy consumption
            energy_consumption = distance * self.robot_params["energy_consumption_rate"]
            battery_level -= energy_consumption

            if battery_level < 0:
                return False  # Energy depletion

        # Check time window for the last node
        last_node = route.nodes[-1]
        if last_node in self.time_windows:
            earliest, latest = self.time_windows[last_node]
            if current_time < earliest:
                current_time = earliest
            elif current_time > latest:
                return False  # Time window violation

        return True

    def calculate_time_warp(self, path: List[int]) -> float:
        """Calculate the time warp between two charging stations."""
        if len(path) < 2:
            return 0

        # Calculate earliest arrival time
        earliest_arrival = 0
        for i in range(len(path) - 1):
            node1, node2 = path[i], path[i+1]
            travel_time = self.distance_matrix[node1][node2] / self.van_params["speed"]
            earliest_arrival += travel_time
            if node1 in self.time_windows:
                earliest, _ = self.time_windows[node1]
                if earliest_arrival < earliest:
                    earliest_arrival = earliest

        # Calculate latest starting time
        latest_start = earliest_arrival
        for i in range(len(path) - 1, 0, -1):
            node1, node2 = path[i-1], path[i]
            travel_time = self.distance_matrix[node1][node2] / self.van_params["speed"]
            latest_start -= travel_time
            if node2 in self.time_windows:
                _, latest = self.time_windows[node2]
                latest_start = min(latest_start, latest - travel_time)

        # Time warp is the difference
        time_warp = latest_start - 0  # Earliest start time is 0
        return max(0, time_warp)

    def create_initial_solution(self) -> "TwoEVREC":
        """
        Create an initial feasible solution using a greedy approach.
        1. Create van routes that visit charging stations
        2. Assign robot routes from charging stations to customers and back
        3. Assign van-customers to van routes where feasible
        """
        solution = self.copy()
        solution.unassigned_customers = set(self.all_customers)

        # Helper function to create a route pair (van and robot)
        def create_route_pair(station: int) -> Tuple[Route, Route]:
            van_route = Route(is_van_route=True)
            van_route.nodes = [self.depot, station, self.depot]
            van_route.robot_onboard = [True, True]  # Robot on board for entire route

            # Set initial recharging amounts
            van_route.recharge_amount[station] = self.van_params["battery_capacity"] / 2

            robot_route = Route(is_van_route=False)
            robot_route.nodes = [station]
            robot_route.recharge_amount[station] = self.robot_params["battery_capacity"]

            return van_route, robot_route

        # Find the closest charging station to the depot
        station_distances = [(s, self.distance_matrix[self.depot][s])
                            for s in self.charging_stations]
        station_distances.sort(key=lambda x: x[1])

        # Create initial routes
        all_assigned = False
        for station, _ in station_distances:
            if all_assigned:
                break

            van_route, robot_route = create_route_pair(station)

            # Try to assign customers to the robot route greedily
            customers_sorted = sorted(
                solution.unassigned_customers,
                key=lambda c: self.distance_matrix[station][c]
            )

            total_load = 0
            robot_customers = []

            for customer in customers_sorted:
                if (total_load + self.customer_demand[customer] <= self.robot_params["capacity"]):
                    # Try with this customer added
                    robot_route.nodes.append(customer)
                    robot_route.nodes.append(station)  # Return to station

                    if solution._is_robot_route_feasible(robot_route, 0):
                        # Customer can be added
                        robot_customers.append(customer)
                        total_load += self.customer_demand[customer]
                        solution.unassigned_customers.remove(customer)
                    else:
                        # Undo the changes
                        robot_route.nodes.pop()
                        robot_route.nodes.pop()

            # Finalize robot route if any customers were assigned
            if robot_customers:
                # Build the route with all customers
                robot_route.nodes = [station]
                for customer in robot_customers:
                    robot_route.nodes.append(customer)
                robot_route.nodes.append(station)

                # Add routes to solution
                solution.van_routes.append(van_route)
                solution.robot_routes.append(robot_route)

            # Check if all customers are assigned
            if not solution.unassigned_customers:
                all_assigned = True

        # Handle any remaining unassigned customers by creating dedicated routes
        for customer in list(solution.unassigned_customers):
            closest_station = min(self.charging_stations,
                                 key=lambda s: self.distance_matrix[s][customer])

            van_route = Route(is_van_route=True)
            van_route.nodes = [self.depot, closest_station, self.depot]
            van_route.robot_onboard = [True, True]
            van_route.recharge_amount[closest_station] = self.van_params["battery_capacity"] / 2

            robot_route = Route(is_van_route=False)
            robot_route.nodes = [closest_station, customer, closest_station]
            robot_route.recharge_amount[closest_station] = self.robot_params["battery_capacity"]

            if solution._is_robot_route_feasible(robot_route, len(solution.robot_routes)):
                solution.van_routes.append(van_route)
                solution.robot_routes.append(robot_route)
                solution.unassigned_customers.remove(customer)

        # Try to optimize by serving some customers directly with vans
        for customer in self.customers_both:
            for i, route in enumerate(solution.van_routes):
                # Check if we can add this customer to a van route
                for pos in range(1, len(route.nodes)):
                    temp_route = route.copy()
                    temp_route.nodes.insert(pos, customer)

                    # Update robot_onboard list
                    temp_route.robot_onboard.insert(pos, temp_route.robot_onboard[pos-1])

                    if solution._is_van_route_feasible(temp_route, i):
                        # Update the route
                        solution.van_routes[i] = temp_route
                        break

        return solution

# Destroy and repair operators for the ALNS algorithm
def random_customer_removal(solution: TwoEVREC, random_state: np.random.RandomState) -> TwoEVREC:
    """
    Remove a random customer from the solution.
    """
    destroyed = solution.copy()
    assigned_customers = [c for c in destroyed.all_customers
                         if c not in destroyed.unassigned_customers]

    if not assigned_customers:
        return destroyed  # No customers to remove

    # Select a destruction rate between 10-30%
    destruction_rate = random_state.uniform(0.1, 0.3)
    num_to_remove = max(1, int(len(assigned_customers) * destruction_rate))

    customers_to_remove = random_state.choice(assigned_customers,
                                             size=min(num_to_remove, len(assigned_customers)),
                                             replace=False)

    for customer in customers_to_remove:
        # Remove from van routes
        for route in destroyed.van_routes:
            if customer in route.nodes:
                idx = route.nodes.index(customer)
                route.nodes.pop(idx)
                if idx < len(route.robot_onboard):
                    route.robot_onboard.pop(idx)

        # Remove from robot routes
        for route in destroyed.robot_routes:
            if customer in route.nodes:
                route.nodes.remove(customer)

        destroyed.unassigned_customers.add(customer)

    # Clean up empty robot routes and their corresponding van routes
    empty_robot_routes = []
    for i, route in enumerate(destroyed.robot_routes):
        if len(route.nodes) <= 2:  # Only has charging stations
            empty_robot_routes.append(i)

    # Remove empty robot routes (in reverse order to avoid index issues)
    for i in sorted(empty_robot_routes, reverse=True):
        destroyed.robot_routes.pop(i)

    return destroyed

def greedy_customer_removal(solution: TwoEVREC, random_state: np.random.RandomState) -> TwoEVREC:
    """
    Remove customers that can yield the largest cost reduction.
    """
    destroyed = solution.copy()
    assigned_customers = [c for c in destroyed.all_customers
                         if c not in destroyed.unassigned_customers]

    if not assigned_customers:
        return destroyed  # No customers to remove

    # Calculate cost impact of removing each customer
    cost_impacts = []
    for customer in assigned_customers:
        # Deep copy to calculate cost difference
        temp_solution = destroyed.copy()

        # Remove customer from van routes
        for route in temp_solution.van_routes:
            if customer in route.nodes:
                idx = route.nodes.index(customer)
                route.nodes.pop(idx)
                if idx < len(route.robot_onboard):
                    route.robot_onboard.pop(idx)

        # Remove from robot routes
        for route in temp_solution.robot_routes:
            if customer in route.nodes:
                route.nodes.remove(customer)

        temp_solution.unassigned_customers.add(customer)

        # Calculate cost improvement
        cost_impact = destroyed.objective() - temp_solution.objective()
        cost_impacts.append((customer, cost_impact))

    # Sort by cost impact (highest reduction first)
    cost_impacts.sort(key=lambda x: x[1], reverse=True)

    # Select a destruction rate between 10-30%
    destruction_rate = random_state.uniform(0.1, 0.3)
    num_to_remove = max(1, int(len(assigned_customers) * destruction_rate))

    # Remove customers with highest cost improvement
    for i in range(min(num_to_remove, len(cost_impacts))):
        customer = cost_impacts[i][0]

        # Remove from van routes
        for route in destroyed.van_routes:
            if customer in route.nodes:
                idx = route.nodes.index(customer)
                route.nodes.pop(idx)
                if idx < len(route.robot_onboard):
                    route.robot_onboard.pop(idx)

        # Remove from robot routes
        for route in destroyed.robot_routes:
            if customer in route.nodes:
                route.nodes.remove(customer)

        destroyed.unassigned_customers.add(customer)

    # Clean up empty robot routes
    empty_robot_routes = []
    for i, route in enumerate(destroyed.robot_routes):
        if len(route.nodes) <= 2:  # Only has charging stations
            empty_robot_routes.append(i)

    # Remove empty robot routes (in reverse order)
    for i in sorted(empty_robot_routes, reverse=True):
        destroyed.robot_routes.pop(i)

    return destroyed

def station_route_removal(solution: TwoEVREC, random_state: np.random.RandomState) -> TwoEVREC:
    """
    Remove a charging station and its associated robot routes.
    """
    destroyed = solution.copy()

    # Find all used stations
    used_stations = set()
    for route in destroyed.robot_routes:
        for node in route.nodes:
            if node in destroyed.charging_stations:
                used_stations.add(node)

    if not used_stations:
        return destroyed  # No stations to remove

    # Randomly select a station to remove
    station_to_remove = random_state.choice(list(used_stations))

    # Identify robot routes that use this station
    routes_to_remove = []
    for i, route in enumerate(destroyed.robot_routes):
        if station_to_remove in route.nodes:
            routes_to_remove.append(i)
            for node in route.nodes:
                if node in destroyed.all_customers:
                    destroyed.unassigned_customers.add(node)

    # Remove robot routes (in reverse order)
    for i in sorted(routes_to_remove, reverse=True):
        destroyed.robot_routes.pop(i)

    # Remove station from van routes
    for route in destroyed.van_routes:
        if station_to_remove in route.nodes:
            # If this is the only station in the route, just empty the route
            if sum(1 for n in route.nodes if n in destroyed.charging_stations) == 1:
                route.nodes = [destroyed.depot, destroyed.depot]
                route.robot_onboard = [False]
                route.recharge_amount = {}
                route.en_route_charge = {}
            else:
                # Remove the station from the route
                indices = [i for i, n in enumerate(route.nodes) if n == station_to_remove]
                for idx in sorted(indices, reverse=True):
                    del route.nodes[idx]
                    if idx < len(route.robot_onboard):
                        del route.robot_onboard[idx]
                if station_to_remove in route.recharge_amount:
                    del route.recharge_amount[station_to_remove]

    return destroyed

def route_destruction(solution: TwoEVREC, random_state: np.random.RandomState) -> TwoEVREC:
    """
    Destroy a random van route and its associated robot routes.
    """
    destroyed = solution.copy()

    if not destroyed.van_routes:
        return destroyed  # No routes to destroy

    # Randomly select a van route to destroy
    route_idx = random_state.randint(0, len(destroyed.van_routes))

    if route_idx < len(destroyed.van_routes):
        # Get the stations in this van route
        van_route = destroyed.van_routes[route_idx]
        stations_in_route = [node for node in van_route.nodes
                            if node in destroyed.charging_stations]

        # Mark all customers in van route as unassigned
        for node in van_route.nodes:
            if node in destroyed.all_customers:
                destroyed.unassigned_customers.add(node)

        # Remove associated robot routes
        routes_to_remove = []
        for i, route in enumerate(destroyed.robot_routes):
            # Check if any station in the van route is in this robot route
            if any(station in route.nodes for station in stations_in_route):
                routes_to_remove.append(i)
                for node in route.nodes:
                    if node in destroyed.all_customers:
                        destroyed.unassigned_customers.add(node)

        # Remove robot routes (in reverse order)
        for i in sorted(routes_to_remove, reverse=True):
            destroyed.robot_routes.pop(i)

        # Remove the van route
        destroyed.van_routes.pop(route_idx)

    return destroyed

def greedy_repair(solution: TwoEVREC, random_state: np.random.RandomState) -> TwoEVREC:
    """
    Repair by greedily inserting unassigned customers into existing routes.
    """
    repaired = solution.copy()

    if not repaired.unassigned_customers:
        return repaired  # Nothing to repair

    unassigned = list(repaired.unassigned_customers)
    random_state.shuffle(unassigned)  # Randomize order

    for customer in unassigned:
        best_cost = float('inf')
        best_insert = None

        # Try to insert into van routes if this is a customer that can be served by vans
        if customer in repaired.customers_both:
            for r_idx, route in enumerate(repaired.van_routes):
                for pos in range(1, len(route.nodes)):  # Skip depot at position 0
                    prev_node, next_node = route.nodes[pos - 1], route.nodes[pos]

                    # Calculate cost delta for this insertion
                    old_cost = (repaired.van_params["travel_cost_rate"] *
                               repaired.distance_matrix[prev_node][next_node])
                    new_cost = (repaired.van_params["travel_cost_rate"] *
                               (repaired.distance_matrix[prev_node][customer] +
                                repaired.distance_matrix[customer][next_node]))
                    delta = new_cost - old_cost

                    if delta < best_cost:
                        # Try inserting and check feasibility
                        temp_route = route.copy()
                        temp_route.nodes.insert(pos, customer)

                        # Update robot_onboard list
                        if pos - 1 < len(route.robot_onboard):
                            rob_onboard = route.robot_onboard[pos - 1]
                            temp_route.robot_onboard.insert(pos, rob_onboard)

                        if repaired._is_van_route_feasible(temp_route, r_idx):
                            best_cost = delta
                            best_insert = ('van', r_idx, pos)

        # Try to insert into robot routes
        for r_idx, route in enumerate(repaired.robot_routes):
            for pos in range(1, len(route.nodes)):  # Skip first station
                # Don't insert between consecutive stations
                if (pos < len(route.nodes) - 1 and
                    route.nodes[pos - 1] in repaired.charging_stations and
                    route.nodes[pos] in repaired.charging_stations):
                    continue

                prev_node, next_node = route.nodes[pos - 1], route.nodes[pos]

                # Calculate cost delta
                old_cost = (repaired.robot_params["travel_cost_rate"] *
                           repaired.distance_matrix[prev_node][next_node])
                new_cost = (repaired.robot_params["travel_cost_rate"] *
                           (repaired.distance_matrix[prev_node][customer] +
                            repaired.distance_matrix[customer][next_node]))
                delta = new_cost - old_cost

                if delta < best_cost:
                    # Try inserting and check feasibility
                    temp_route = route.copy()
                    temp_route.nodes.insert(pos, customer)

                    if repaired._is_robot_route_feasible(temp_route, r_idx):
                        best_cost = delta
                        best_insert = ('robot', r_idx, pos)

        if best_insert:
            # Apply the best insertion
            route_type, r_idx, pos = best_insert
            if route_type == 'van':
                # Insert into van route
                repaired.van_routes[r_idx].nodes.insert(pos, customer)
                if pos - 1 < len(repaired.van_routes[r_idx].robot_onboard):
                    rob_onboard = repaired.van_routes[r_idx].robot_onboard[pos - 1]
                    repaired.van_routes[r_idx].robot_onboard.insert(pos, rob_onboard)
            else:  # robot
                # Insert into robot route
                repaired.robot_routes[r_idx].nodes.insert(pos, customer)

            # Mark as assigned
            repaired.unassigned_customers.remove(customer)
        else:
            # Create a new route pair for this customer if insertion failed
            # Find the closest charging station
            closest_station = min(repaired.charging_stations,
                                 key=lambda s: repaired.distance_matrix[s][customer])

            # Create new van and robot routes
            van_route = Route(is_van_route=True)
            van_route.nodes = [repaired.depot, closest_station, repaired.depot]
            van_route.robot_onboard = [True, True]
            van_route.recharge_amount[closest_station] = repaired.van_params["battery_capacity"] / 2

            robot_route = Route(is_van_route=False)
            robot_route.nodes = [closest_station, customer, closest_station]
            robot_route.recharge_amount[closest_station] = repaired.robot_params["battery_capacity"]

            # Check feasibility and add routes
            if (repaired._is_van_route_feasible(van_route, len(repaired.van_routes)) and
                repaired._is_robot_route_feasible(robot_route, len(repaired.robot_routes))):
                repaired.van_routes.append(van_route)
                repaired.robot_routes.append(robot_route)
                repaired.unassigned_customers.remove(customer)

    return repaired

def route_reconstruction(solution: TwoEVREC, random_state: np.random.RandomState) -> TwoEVREC:
    """
    Reconstruct routes for all unassigned customers.
    """
    reconstructed = solution.copy()

    if not reconstructed.unassigned_customers:
        return reconstructed  # Nothing to reconstruct

    unassigned = list(reconstructed.unassigned_customers)

    # Sort by demand (descending) to prioritize harder-to-place customers
    unassigned.sort(key=lambda c: reconstructed.customer_demand[c], reverse=True)

    for customer in unassigned:
        # Try creating a direct van route for van-accessible customers
        if customer in reconstructed.customers_both:
            # Create direct van route
            van_route = Route(is_van_route=True)
            van_route.nodes = [reconstructed.depot, customer, reconstructed.depot]
            van_route.robot_onboard = [False, False]

            if reconstructed._is_van_route_feasible(van_route, len(reconstructed.van_routes)):
                reconstructed.van_routes.append(van_route)
                reconstructed.unassigned_customers.remove(customer)
                continue

        # Otherwise, create a robot route via a charging station
        # Find all charging stations ordered by distance
        station_distances = [(s, reconstructed.distance_matrix[customer][s])
                            for s in reconstructed.charging_stations]
        station_distances.sort(key=lambda x: x[1])

        # Try each station until we find a feasible route
        for station, _ in station_distances:
            # Create new van and robot routes
            van_route = Route(is_van_route=True)
            van_route.nodes = [reconstructed.depot, station, reconstructed.depot]
            van_route.robot_onboard = [True, True]
            van_route.recharge_amount[station] = reconstructed.van_params["battery_capacity"] / 2

            robot_route = Route(is_van_route=False)
            robot_route.nodes = [station, customer, station]
            robot_route.recharge_amount[station] = reconstructed.robot_params["battery_capacity"]

            # Set en-route charging amount
            distance_to_customer = reconstructed.distance_matrix[station][customer]
            energy_needed = distance_to_customer * reconstructed.robot_params["energy_consumption_rate"] * 2  # Round trip

            # Determine if en-route charging is needed
            if energy_needed > reconstructed.robot_params["battery_capacity"]:
                van_route.en_route_charge[(reconstructed.depot, station)] = energy_needed / 2

            # Check feasibility and add routes
            if (reconstructed._is_van_route_feasible(van_route, len(reconstructed.van_routes)) and
                reconstructed._is_robot_route_feasible(robot_route, len(reconstructed.robot_routes))):
                reconstructed.van_routes.append(van_route)
                reconstructed.robot_routes.append(robot_route)
                reconstructed.unassigned_customers.remove(customer)
                break

    return reconstructed

def optimize_energy(solution: TwoEVREC, random_state: np.random.RandomState) -> TwoEVREC:
    """
    Optimize energy usage by adjusting charging amounts and applying en-route charging.
    """
    optimized = solution.copy()

    # For each van route, adjust charging amounts
    for i, van_route in enumerate(optimized.van_routes):
        if len(van_route.nodes) <= 2:  # Skip empty routes
            continue

        for j in range(len(van_route.nodes) - 1):
            node1, node2 = van_route.nodes[j], van_route.nodes[j + 1]

            # If robot is on board, consider en-route charging
            if j < len(van_route.robot_onboard) and van_route.robot_onboard[j]:
                # Calculate energy needed for the robot's next segment
                for robot_route in optimized.robot_routes:
                    # Find the robot route that starts at this station
                    if len(robot_route.nodes) > 0 and robot_route.nodes[0] == node2 and node2 in optimized.charging_stations:
                        # Calculate robot energy needs
                        total_energy_needed = 0
                        for k in range(len(robot_route.nodes) - 1):
                            r_node1, r_node2 = robot_route.nodes[k], robot_route.nodes[k + 1]
                            distance = optimized.distance_matrix[r_node1][r_node2]
                            energy_needed = distance * optimized.robot_params["energy_consumption_rate"]
                            total_energy_needed += energy_needed

                        # Apply en-route charging if needed
                        travel_time = optimized.distance_matrix[node1][node2] / optimized.van_params["speed"]
                        max_charge = travel_time * optimized.robot_params["charge_rate"]

                        # Determine optimal charging amount
                        charging_amount = min(max_charge, total_energy_needed,
                                            optimized.robot_params["battery_capacity"])

                        if charging_amount > 0:
                            van_route.en_route_charge[(node1, node2)] = charging_amount
                            break

    # For each charging station, optimize charging amounts
    for van_route in optimized.van_routes:
        for node in van_route.nodes:
            if node in optimized.charging_stations:
                # Calculate energy needed to reach the next station or depot
                energy_needed = 0
                station_idx = van_route.nodes.index(node)

                for j in range(station_idx, len(van_route.nodes) - 1):
                    n1, n2 = van_route.nodes[j], van_route.nodes[j + 1]
                    distance = optimized.distance_matrix[n1][n2]
                    energy_used = distance * optimized.van_params["energy_consumption_rate"]
                    energy_needed += energy_used

                    # Add energy needed for en-route charging
                    en_route_charge = van_route.en_route_charge.get((n1, n2), 0)
                    energy_needed += en_route_charge

                    # Stop if we reach another charging station
                    if n2 in optimized.charging_stations:
                        break

                # Set optimal charging amount
                van_route.recharge_amount[node] = min(energy_needed,
                                                    optimized.van_params["battery_capacity"])

    return optimized

def main():
    # Set seed for reproducibility
    SEED = 1234
    random.seed(SEED)
    np.random.seed(SEED)

    # Problem data
    distance_matrix = np.array([
        [0, 50, 80, 80, 50, 80, 80, 82, 84, 86],  # depot 0
        [50, 0, 70, 80, 80, 80, 50, 50, 90, 90],  # station 1
        [80, 70, 0, 50, 80, 80, 80, 80, 80, 80],  # station 2
        [80, 80, 50, 0, 50, 80, 50, 50, 80, 80],  # station 3
        [50, 80, 80, 50, 0, 70, 90, 80, 30, 30],  # station 4
        [80, 80, 80, 80, 70, 0, 80, 80, 80, 80],  # station 5
        [80, 50, 80, 50, 90, 80, 0, 80, 90, 90],  # customer 6
        [82, 50, 80, 50, 80, 80, 80, 0, 90, 90],  # customer 7
        [84, 90, 80, 80, 30, 80, 90, 90, 0, 30],  # customer 8
        [86, 90, 80, 80, 30, 80, 90, 90, 30, 0]   # customer 9
    ])

    depot = 0
    charging_stations = [1, 2, 3, 4, 5]
    customers_robot_only = [6, 8, 9]
    customers_both = [7]

    van_params = {
        "speed": 2.0,
        "battery_capacity": 400.0,
        "capacity": 200.0,
        "travel_cost_rate": 2.0,
        "energy_consumption_rate": 2.0,
        "charge_rate": 10.0
    }

    robot_params = {
        "speed": 1.0,
        "battery_capacity": 120.0,
        "capacity": 30.0,
        "travel_cost_rate": 1.0,
        "energy_consumption_rate": 1.0,
        "charge_rate": 4.0
    }

    customer_demand = {
        6: 15,
        7: 20,
        8: 10,
        9: 25
    }

    # Optional: time windows (depot open 0-8, customers have 2h windows)
    time_windows = {
        0: (0, 8),  # Depot
        7: (1, 3),  # Sample time windows for customers
        6: (2, 4),
        8: (3, 5),
        9: (4, 6)
    }

    # Initialize problem
    problem = TwoEVREC(
        distance_matrix=distance_matrix,
        depot=depot,
        charging_stations=charging_stations,
        customers_robot_only=customers_robot_only,
        customers_both=customers_both,
        van_params=van_params,
        robot_params=robot_params,
        customer_demand=customer_demand,
        time_windows=time_windows
    )

    # Create initial solution
    initial_solution = problem.create_initial_solution()
    print(f"Initial solution objective value: {initial_solution.objective()}")
    print(f"Initial solution feasible: {initial_solution.is_feasible()}")

    # Set up ALNS algorithm
    alns = ALNS(rnd.default_rng(SEED))

    # Add destroy operators
    alns.add_destroy_operator(random_customer_removal)
    alns.add_destroy_operator(greedy_customer_removal)
    alns.add_destroy_operator(station_route_removal)
    alns.add_destroy_operator(route_destruction)

    # Add repair operators
    alns.add_repair_operator(greedy_repair)
    alns.add_repair_operator(route_reconstruction)
    alns.add_repair_operator(optimize_energy)

    # Configure selection, acceptance, and stopping criteria
    select = RouletteWheel([25, 5, 1, 0], 0.8, 1, 1)
    accept = SimulatedAnnealing(
        start_temperature=100,
        end_temperature=1,
        step=0.99,
        method="exponential"
    )
    stop = MaxIterations(1000)

    # Run ALNS
    result = alns.iterate(initial_solution, select, accept, stop)
    best_solution = result.best_state

    # Output results
    print("\nBest solution:")
    print(f"Objective value: {best_solution.objective()}")
    print(f"Feasible: {best_solution.is_feasible()}")

    print("\nVan Routes:")
    for i, route in enumerate(best_solution.van_routes):
        print(f"  Route {i + 1}: {route.nodes}")
        print(f"    Robot onboard: {route.robot_onboard}")
        print(f"    Recharge amounts: {route.recharge_amount}")
        print(f"    En-route charging: {route.en_route_charge}")

    print("\nRobot Routes:")
    for i, route in enumerate(best_solution.robot_routes):
        print(f"  Route {i + 1}: {route.nodes}")
        print(f"    Recharge amounts: {route.recharge_amount}")

    # Print statistics
    print("\nALNS Statistics:")
    print(f"Total iterations: {result.statistics.iterations}")
    print(f"Best objective: {result.statistics.best_objectives[-1]}")
    print(f"Current objective: {result.statistics.current_objectives[-1]}")

if __name__ == "__main__":
    main()

Initial solution objective value: 0.0
Initial solution feasible: False

Best solution:
Objective value: 0.0
Feasible: False

Van Routes:

Robot Routes:

ALNS Statistics:


AttributeError: 'Statistics' object has no attribute 'iterations'